In [1]:
import matplotlib.pyplot as plt
import numpy as np
import gvar as gv
import pprint

In [2]:
import re


def parse_data(input_data):
    data = {}

    # Define the order of the baryons
    baryon_order = ["N_E0", "Xi_EO", "Sigma_E0",
                    "Lambda_E0", "Xi*_E0", "Sigma*_E0"]

    # Define a mapping from your original baryon names to the new ones
    baryon_mapping = {'lambda_z': 'Lambda_E0',
                      'proton': 'N_E0',
                      'sigma_p': 'Sigma_E0',
                      'sigma_star_p': 'Sigma*_E0',
                      'xi_star_z': 'Xi*_E0',
                      'xi_z': 'Xi_EO'}

    for key, value in input_data.items():
        ensemble_name, baryon = key
        number, uncertainty = gv.mean(value), gv.sdev(value)*10000

        # ensemble_name = ens_map.get(ensemble_name,ensemble_name)

        if ensemble_name not in data:
            data[ensemble_name] = {}

        # Map the original baryon name to the new one
        new_baryon_name = baryon_mapping[baryon]
        data[ensemble_name][new_baryon_name] = (number, uncertainty)
        
    for ensemble_name in data:
        data[ensemble_name] = {baryon: data[ensemble_name].get(
            baryon, (None, None)) for baryon in baryon_order}


    return data


In [11]:
nolan = {}
print(0.0022 * 10000)
nolan = gv.load('avg_E0.p')
nolan_data = parse_data(nolan)


22.0


In [10]:
import re  
import pprint

def parse_latex_table(file_name):
    with open(file_name, "r") as f:
        content = f.read()

    # tables = content.split("\n\n")
    tables = re.split(r'\\end{tabular}', content)[:-1]

    data = {}
    for table in tables:
        lines = table.strip().split("\n")
        if len(lines) < 4:  # If there are not enough lines, skip this table
            continue
        ensemble = lines[2].split("&")[0].strip().replace('$_','_').rstrip('$')
        data[ensemble] = {}

        for line in lines[3:-1]:  # Skip the first two lines (headers) and the last line (end of the table)
            cols = line.split("&")

            person = cols[0].strip()
            results = [tuple(map(float, re.findall(r"([\d.]+)", item))) for item in cols[1:]]
            data[ensemble][person] = dict(zip(["N_E0", "Xi_EO", "Sigma_E0", "Lambda_E0", "Xi*_E0", "Delta_E0", "Sigma*_E0"], results))

    return data

file_name = "tables.txt"
baryon_order = ["N_E0", "Xi_EO", "Sigma_E0",
                "Lambda_E0", "Xi*_E0", "Delta_E0", "Sigma*_E0"]
data = parse_latex_table(file_name)
for ensemble_name in data:
    data[ensemble_name]['nolan'] = nolan_data.get(ensemble_name, {baryon: (None, None) for baryon in baryon_order})
# pprint.pprint(data)
data.keys()



dict_keys(['a06m220L', 'a06m310L', 'a09m135', 'a09m220_o', 'a09m260', 'a09m310_o', 'a09m350', 'a09m400', 'a12m130', 'a12m180L', 'a12m220L', 'a12m220S', 'a12m220_ms', 'a12m220_o', 'a12m260', 'a12m310', 'a12m310L', 'a12m310XL', 'a12m350', 'a12m400', 'a15m130', 'a15m135XL', 'a15m220', 'a15m260', 'a15m310', 'a15m310L_o', 'a15m350', 'a15m400'])

In [9]:

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

def plot_ensemble(ensemble_data, ensemble_name,pdf):
    fig, ax = plt.subplots(figsize=(12, 8))

    offset = 0.05
    all_baryon_keys = set()

    for i, (person, results) in enumerate(ensemble_data.items()):
        if person == '\\hline':
            continue
        # valid_results = {k: v for k, v in results.items() if len(v) > 0}
        valid_results = {k: v for k, v in results.items() if v and len(v)>1 and v[0] is not None}
        # print(f"Valid results for {person}: {valid_results}")
        # valid_results = {k: v for k, v in results.items() if v and len(v) > 1 }

        x = np.arange(len(valid_results))
        y = [result[0] for result in valid_results.values()]
        # yerr = [result[1] *0.0001 if len(result) > 1 else 0 for result in valid_results.values()]
        yerr = [result[1] * 0.0001 if result[1] is not None else 0 for result in valid_results.values()]

        ax.errorbar(x + i * offset, y, yerr=yerr, fmt="o", capsize=5, label=person)

        all_baryon_keys.update(valid_results.keys())

    all_keys = sorted(list(all_baryon_keys))  # Convert to a list and sort it
    ax.set_xticks(np.arange(len(all_keys)) +
                  (len(ensemble_data) - 1) * offset / 2)
    ax.set_xticklabels(all_keys)
    ax.set_xlim(-0.5, len(all_keys) - 0.5 + (len(ensemble_data) - 1) * offset)

    ax.legend()
    ax.set_xlabel("Baryon")
    ax.set_ylabel("Fit Result")
    ax.set_title(f"Fit Results for {ensemble_name}")

    pdf.savefig(fig)
    plt.close(fig)

with PdfPages("output_plots.pdf") as pdf:
    for ensemble_name, ensemble_data in data.items():
        plot_ensemble(ensemble_data, ensemble_name, pdf)


